In [6]:
!pip install kaggle

In [7]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 94.5 MB/s eta 0:00:00


In [8]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [9]:
import torch
from torch import nn
import timm
from transformers import DistilBertModel , DistilBertConfig , DistilBertTokenizer
import torch.nn.functional as F
import albumentations as A
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import itertools

In [10]:
class Configure():

  #For getting images
  size = 224
  image_path = '/content/drive/MyDrive/flickr8k/Images'
  captions_path = '/content/drive/MyDrive/flickr8k'


  dim = 256
  #for caption tokenizer
  max_length = 200


  #for text and image encoder
  image_model = 'resnet50'
  image_embedding = 2048
  text_model = 'distilbert-base-uncased'
  text_embedding = 768
  text_tokenizer = 'distilbert-base-uncased'

  pretrained = True
  trainable = True

  #For projection head
  projection_dim = 256
  dropout = 0.1
  num_projection_layers = 1


  #For CLIP model
  temperature = 1.0

  batch_size = 32
  num_workers = 4
  head_lr = 1e-3
  image_encoder_lr = 1e-4
  text_encoder_lr = 1e-5
  weight_decay = 1e-3
  patience = 1
  factor = 0.8
  epochs = 4
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  debug = False

In [11]:
class AvgMeter:
    def __init__(self, name="Metric"):
        self.name = name
        self.reset()

    def reset(self):
        self.avg, self.sum, self.count = [0] * 3

    def update(self, val, count=1):
        self.count += count
        self.sum += val * count
        self.avg = self.sum / self.count

    def __repr__(self):
        text = f"{self.name}: {self.avg:.4f}"
        return text

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group["lr"]

In [12]:
#image_filenames and captions must have same length so if there are multiple captions of same image image_filename must be same for all.


class CLIPdata(torch.utils.data.Dataset):

  def __init__(self,image_filenames,captions,tokenizer,transform):
    self.image_filenames = image_filenames
    self.captions = list(captions)
    self.encoded_options = tokenizer(list(captions),padding=True,truncation=True,max_length = Configure.max_length)
    self.transform = transform


  def __getitem__(self, idx):
        item = {
            key: torch.tensor(values[idx])
            for key, values in self.encoded_captions.items()
        }

        image = cv2.imread(f"{Configure.image_path}/{self.image_filenames[idx]}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transforms(image=image)['image']
        item['image'] = torch.tensor(image).permute(2, 0, 1).float()
        item['caption'] = self.captions[idx]

        return item

  def __len__(self):
      return len(self.captions)


# def get_transforms(mode ='train'):
#       if mode == 'train':
#         return A.compose([A.resize(Configure.size,Configure.size,always_apply = True),A.normalize(max_pixel_value=255.0,always_apply = True)])
#       else:
#         return A.compose([A.resize(Configure.size,Configure.size,always_apply = True),A.normalize(max_pixel_value=255.0,always_apply = True)])
def get_transforms(mode='train'):
    if mode == 'train':
        return A.Compose([
            A.Resize(Configure.size, Configure.size, always_apply=True),
            A.Normalize(max_pixel_value=255.0, always_apply=True)
        ])
    else:
        return A.Compose([
            A.Resize(Configure.size, Configure.size, always_apply=True),
            A.Normalize(max_pixel_value=255.0, always_apply=True)
        ])

In [13]:
#Build Image Encoder
class ImageEncoder(nn.Module):

  def __init__(self,image_model=Configure.image_model,pretrained = Configure.pretrained,trainable = Configure.trainable):
    self.model = timm.create_model(image_model,pretrained,global_pool='avg')

  def forward(self,x):
    return self.model(x)

In [14]:
#Build Text Encoder

class TextEncoder(nn.Module):

  def __init__(self,text_model = Configure.text_model,pretrained = Configure.pretrained):

    self.model = DistilBertModel.from_pretrained(text_model)
    self.target_token_idx = 0



  def forward(self,input_ids,attention_mask):
    output = self.model(input_ids = input_ids,attention_mask=attention_mask)
    last_hidden_state = output.last_hidden_state
    return last_hidden_state[:,self.target_token_idx,:]

In [15]:
#Bring both embeddings into same dimensions(Image and text)

class ProjectHead(nn.Module):

  def __init__(self,embedding_dim,projection_dim=Configure.projection_dim,dropout = Configure.dropout):
          self.projection = nn.Linear(embedding_dim,projection_dim) #embedding_dim is size of i/p vector(2048 for images and 768 for text) and projection_dim is o/p vector of size 256
          self.gelu = nn.GELU()
          self.fc = nn.Linear(projection_dim,projection_dim)
          self.dropout = nn.Dropout(dropout)
          self.layer_norm = nn.LayerNorm(projection_dim)

  def forward(self,x):
        projected = self.projection(x)
        x = self.gelu(projected)
        x = self.fc(x)
        x = self.dropout(x)
        x = x + projected
        x = self.layer_norm(x)
        return x

In [21]:
class CLIPModel(nn.Module):

  # def __init__(self,temperature=Configure.temperature,image_embedding = Configure.image_embedding,text_embedding = Configure.text_embedding):
  #   self.image_encoder = ImageEncoder()
  #   self.text_encoder = TextEncoder()
  #   self.image_projection = ProjectHead(embedding_dim=image_embedding)
  #   self.text_projection = ProjectHead(embedding_dim = text_embedding)
  #   self.temperature = temperature
  def __init__(self, temperature=Configure.temperature, image_embedding=Configure.image_embedding, text_embedding=Configure.text_embedding):
        super(CLIPModel, self).__init__()  # Call the parent class constructor
        self.image_encoder = ImageEncoder()  # Create an instance of ImageEncoder
        self.text_encoder = TextEncoder()
        self.image_projection = ProjectHead(embedding_dim=image_embedding)
        self.text_projection = ProjectHead(embedding_dim=text_embedding)
        self.temperature = temperature

  def cross_entropy(self, preds, targets, reduction='none'):
        log_softmax = nn.LogSoftmax(dim=-1)
        loss = (-targets * log_softmax(preds)).sum(1)
        if reduction == "none":
            return loss
        elif reduction == "mean":
            return loss.mean()


  def forward(self,batch):
    #Get image and text features
    image_features = self.image_encoder(batch['image'])
    text_features =  self.text_encoder(input_ids = batch['input_ids'],attention_mask = batch['attention_mask'])

    #Get image and text embeedings(with same dim)
    image_embeddings = self.image_projection(image_features)
    text_embeddings = self.text_projection(text_features)

    #calculate loss
    logits = (text_embeddings @ image_embeddings.T) / self.temperature
    image_similarity = image_embeddings @ image_embeddings.T
    text_similarity = text_similarity @text_similarity.T

    target = F.softmax((image_similarity + text_similarity)/2 * self.temperature,dim= -1)

    text_loss = self.cross_entropy(logits,target,reduction = 'None')
    image_loss = self.cross_entropy(logits.T,target.T,reduction = 'None')
    loss = image_loss + text_loss

    return loss.mean()

In [42]:
import torch
from torch import nn
import torch.nn.functional as F



class CLIPModel(nn.Module):
    def __init__(
        self,
        temperature=Configure.temperature,
        image_embedding=Configure.image_embedding,
        text_embedding=Configure.text_embedding,
    ):
        super().__init__()
        self.image_encoder = ImageEncoder()
        self.text_encoder = TextEncoder()
        self.image_projection = ProjectHead(embedding_dim=image_embedding)
        self.text_projection = ProjectHead(embedding_dim=text_embedding)
        self.temperature = temperature

    def cross_entropy(preds, targets, reduction='none'):
        log_softmax = nn.LogSoftmax(dim=-1)
        loss = (-targets * log_softmax(preds)).sum(1)
        if reduction == "none":
            return loss
        elif reduction == "mean":
            return loss.mean()

    def forward(self, batch):
        # Getting Image and Text Features
        image_features = self.image_encoder(batch["image"])
        text_features = self.text_encoder(
            input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
        )
        # Getting Image and Text Embeddings (with same dimension)
        image_embeddings = self.image_projection(image_features)
        text_embeddings = self.text_projection(text_features)

        # Calculating the Loss
        logits = (text_embeddings @ image_embeddings.T) / self.temperature
        images_similarity = image_embeddings @ image_embeddings.T
        texts_similarity = text_embeddings @ text_embeddings.T
        targets = F.softmax(
            (images_similarity + texts_similarity) / 2 * self.temperature, dim=-1
        )
        texts_loss = self.cross_entropy(logits, targets, reduction='none')
        images_loss =self.cross_entropy(logits.T, targets.T, reduction='none')
        loss =  (images_loss + texts_loss) / 2.0 # shape: (batch_size)
        return loss.mean()

In [29]:
batch_size = 4
dim = 256
embeddings = torch.randn(batch_size, dim)
out = embeddings @ embeddings.T
print(F.softmax(out, dim=-1))

tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])


In [30]:
def make_train_valid_dfs():
    dataframe = pd.read_csv(f"{Configure.captions_path}/captions.csv")
    # max_id = dataframe["id"].max() + 1 if not Configure.debug else 100
    # image_ids = np.arange(0, max_id)
    # np.random.seed(42)
    # valid_ids = np.random.choice(
    #     image_ids, size=int(0.2 * len(image_ids)), replace=False
    # )
    # train_ids = [id_ for id_ in image_ids if id_ not in valid_ids]
    # train_dataframe = dataframe[dataframe["id"].isin(train_ids)].reset_index(drop=True)
    # valid_dataframe = dataframe[dataframe["id"].isin(valid_ids)].reset_index(drop=True)
    train_dataframe = dataframe.sample(frac=0.8, random_state=42)
    valid_dataframe = dataframe.drop(train_dataframe.index)

    return train_dataframe, valid_dataframe


# def build_loaders(dataframe, tokenizer, mode):
#     transforms = get_transforms(mode=mode)
#     dataset = CLIPdata(
#         dataframe["image"].values,
#         dataframe["caption"].values,
#         tokenizer=tokenizer,
#         transforms=transforms,
#     )
#     dataloader = torch.utils.data.DataLoader(
#         dataset,
#         batch_size=Configure.batch_size,
#         num_workers=Configure.num_workers,
#         shuffle=True if mode == "train" else False,
#     )
#     return dataloader

def build_loaders(dataframe, tokenizer, mode):
    transform = get_transforms(mode=mode)  # Use singular 'transform'
    dataset = CLIPdata(
        dataframe["image"].values,
        dataframe["caption"].values,
        tokenizer=tokenizer,
        transform=transform,  # Use 'transform' instead of 'transforms'
    )
    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=Configure.batch_size,
        num_workers=Configure.num_workers,
        shuffle=True if mode == "train" else False,
    )
    return dataloader

In [31]:
def train_epoch(model, train_loader, optimizer, lr_scheduler, step):
    loss_meter = AvgMeter()
    tqdm_object = tqdm(train_loader, total=len(train_loader))
    for batch in tqdm_object:
        batch = {k: v.to(Configure.device) for k, v in batch.items() if k != "caption"}
        loss = model(batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if step == "batch":
            lr_scheduler.step()

        count = batch["image"].size(0)
        loss_meter.update(loss.item(), count)

        tqdm_object.set_postfix(train_loss=loss_meter.avg, lr=get_lr(optimizer))
    return loss_meter


def valid_epoch(model, valid_loader):
    loss_meter = AvgMeter()

    tqdm_object = tqdm(valid_loader, total=len(valid_loader))
    for batch in tqdm_object:
        batch = {k: v.to(Configure.device) for k, v in batch.items() if k != "caption"}
        loss = model(batch)

        count = batch["image"].size(0)
        loss_meter.update(loss.item(), count)

        tqdm_object.set_postfix(valid_loss=loss_meter.avg)
    return loss_meter

def main():
    train_df, valid_df = make_train_valid_dfs()
    tokenizer = DistilBertTokenizer.from_pretrained(Configure.text_tokenizer)
    train_loader = build_loaders(train_df, tokenizer, mode="train")
    valid_loader = build_loaders(valid_df, tokenizer, mode="valid")


    model = CLIPModel().to(Configure.device)
    params = [
        {"params": model.image_encoder.parameters(), "lr": Configure.image_encoder_lr},
        {"params": model.text_encoder.parameters(), "lr": Configure.text_encoder_lr},
        {"params": itertools.chain(
            model.image_projection.parameters(), model.text_projection.parameters()
        ), "lr": Configure.head_lr, "weight_decay": Configure.weight_decay}
    ]
    optimizer = torch.optim.AdamW(params, weight_decay=0.)
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="min", patience=Configure.patience, factor=Configure.factor
    )
    step = "epoch"

    best_loss = float('inf')
    for epoch in range(Configure.epochs):
        print(f"Epoch: {epoch + 1}")
        model.train()
        train_loss = train_epoch(model, train_loader, optimizer, lr_scheduler, step)
        model.eval()
        with torch.no_grad():
            valid_loss = valid_epoch(model, valid_loader)

        if valid_loss.avg < best_loss:
            best_loss = valid_loss.avg
            torch.save(model.state_dict(), "best.pt")
            print("Saved Best Model!")

        lr_scheduler.step(valid_loss.avg)

In [44]:
main()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


AttributeError: ignored

In [40]:
def get_image_embeddings(valid_df, model_path):
    tokenizer = DistilBertTokenizer.from_pretrained(Configure.text_tokenizer)
    valid_loader = build_loaders(valid_df, tokenizer, mode="valid")

    model = CLIPModel().to(Configure.device)
    model.load_state_dict(torch.load(model_path, map_location=Configure.device))
    model.eval()

    valid_image_embeddings = []
    with torch.no_grad():
        for batch in tqdm(valid_loader):
            image_features = model.image_encoder(batch["image"].to(Configure.device))
            image_embeddings = model.image_projection(image_features)
            valid_image_embeddings.append(image_embeddings)
    return model, torch.cat(valid_image_embeddings)

In [41]:
_, valid_df = make_train_valid_dfs()
model, image_embeddings = get_image_embeddings(valid_df, "best.pt")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


AttributeError: ignored

In [ ]:
def find_matches(model, image_embeddings, query, image_filenames, n=9):
    tokenizer = DistilBertTokenizer.from_pretrained(Configure.text_tokenizer)
    encoded_query = tokenizer([query])
    batch = {
        key: torch.tensor(values).to(Configure.device)
        for key, values in encoded_query.items()
    }
    with torch.no_grad():
        text_features = model.text_encoder(
            input_ids=batch["input_ids"], attention_mask=batch["attention_mask"]
        )
        text_embeddings = model.text_projection(text_features)

    image_embeddings_n = F.normalize(image_embeddings, p=2, dim=-1)
    text_embeddings_n = F.normalize(text_embeddings, p=2, dim=-1)
    dot_similarity = text_embeddings_n @ image_embeddings_n.T

    values, indices = torch.topk(dot_similarity.squeeze(0), n * 5)
    matches = [image_filenames[idx] for idx in indices[::5]]

    _, axes = plt.subplots(3, 3, figsize=(10, 10))
    for match, ax in zip(matches, axes.flatten()):
        image = cv2.imread(f"{Configure.image_path}/{match}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        ax.imshow(image)
        ax.axis("off")

    plt.show()

In [ ]:
find_matches(model,
             image_embeddings,
             query="riverside photo",
             image_filenames=valid_df['image'].values,
             n=9)

NameError: ignored